In [6]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.stats.mstats import gmean
from itertools import combinations
from matplotlib import pyplot as plt

In [26]:
def define_outlier_element(D, method):
    n = D.shape[0]

    if method == "shortened":

        def get_CI(D, n):
            w, v = np.linalg.eig(D)
            lambda_max = np.max(w).astype(np.float32)
            return (lambda_max - n) / (n - 1)

        cid = []
        for i in range(D.shape[0]):
            indices = list(range(0, i)) + list(range(i + 1, matrix.shape[0]))
            short = matrix[indices, :][:, indices]
            cid.append(get_CI(short, n - 1))
            print(f"{i} CI: {cid[-1]}")

        cid = np.array(cid)
        j, i = np.argpartition(cid, 2)[:2]
        return np.array([i, j])

    if method == "correlation":
        corrs_i = np.array(
            [
                np.mean([np.corrcoef(D[i], D[j]) for j in range(n) if i != j])
                for i in range(n)
            ]
        )
        corrs_j = np.array(
            [
                np.mean([np.corrcoef(D[:, i], D[:, j]) for j in range(n) if i != j])
                for i in range(n)
            ]
        )
        i = np.argmin(corrs_i)
        j = np.argmin(corrs_j)
        return np.array([i, j])

    if method == "chi":
        T = np.array(
            [
                [(D[i].sum() * D[:, j].sum()) / (D.sum()) for j in range(n)]
                for i in range(n)
            ]
        )
        deltas = ((D - T) ** 2) / T
        mean = deltas.mean()
        sigma = deltas.std()
        deltas_mask = np.array(
            [
                [
                    1 if not (mean - sigma < deltas[i, j] < mean + sigma) else 0
                    for j in range(n)
                ]
                for i in range(n)
            ]
        )
        return np.argwhere(deltas_mask)

    if method == "transitive":
        # He = [np.array([D[i,j], D[j,k], D[i,k]]) for i in range(n) for j in range(n) for k in range(n) if i<j<k]
        # Det = [(D[i,j]*D[j,k]/D[i,k]) + (D[i,k]/(D[i,j]*D[j,k])) - 2 for i in range(n) for j in range(n) for k in range(n) if i<j<k]
        S_ij = np.array(
            [
                (
                    (D[i, j] * D[j, :] / D[i, :]) + (D[i, :] / (D[i, j] * D[j, :])) - 2
                ).sum()
                for i in range(n)
                for j in range(n)
                if i < j
            ]
        )
        indices = [np.array([i, j]) for i in range(n) for j in range(n) if i < j][
            S_ij.argmax()
        ]
        return indices
    
    if method == "outflow":
        F_plus = np.array([(D[i, :] > 1).sum() for i in range(n)])
        F_minus = np.array([(D[:, i] > 1).sum() for i in range(n)])
        print(f"F_plus: {F_plus}")
        print(f"F_minus: {F_minus}")
        res = np.empty((n, n))
        for i in range(n):
            for j in range(n):
                res[i, j] = max(F_plus[j] - F_plus[i], F_minus[i] - F_minus[j])

        if len(np.argwhere(res == res.max())) <= 1:
            return np.argwhere(res == res.max())
        else:
            print("2+ elements satisfy ")
            gamma = {}
            for i in range(n):
                for j in range(n):
                    gamma[(i,j)] = 0
                    for k in range(n):
                        if (i != j and j != k and i != k):
                            gamma[(i,j)]+=np.log(D[i,j]) - np.log(D[i,k]*D[k,j])
                    gamma[(i,j)] /= (n-2)

            result = max(gamma, key=lambda key: gamma[key])
            return np.array(result)
        
    raise ValueError("Method should be one of: 'outflow', 'transitive', 'chi', 'correlation', 'shortened'.")
    

In [27]:
matrix = np.array([[1, 1, 1/2, 4, 3, 1/3, 2],
                   [1, 1, 1/2, 4, 3, 1/3, 2],
                   [2, 7, 1, 8, 6, 1/2, 4],
                   [1/4, 1/4, 1/8, 1, 1, 7, 1/2],
                   [1/3, 1/3, 1/6, 1, 1, 1/9, 1],
                   [3, 3, 2, 1/7, 9, 1, 6],
                   [1/2, 1/2, 1/4, 2, 1, 1/6, 1]])

define_outlier_element(matrix, 'shortened')

0 CI: 0.7903690338134766
1 CI: 0.7279794692993165
2 CI: 0.6933403015136719
3 CI: 0.08690452575683594
4 CI: 0.810521125793457
5 CI: 0.08439168930053711
6 CI: 0.7865217208862305


<ipython-input-26-1c5ccba42b5a>:8: ComplexWarning: Casting complex values to real discards the imaginary part
  lambda_max = np.max(w).astype(np.float32)


array([3, 5])

In [28]:
define_outlier_element(matrix, "correlation")

array([3, 5])

In [29]:
define_outlier_element(matrix, "chi")

array([[3, 5]])

In [30]:
define_outlier_element(matrix, "transitive")

array([3, 5])

In [31]:
define_outlier_element(matrix, "outflow")

F_plus: [3 3 5 1 0 5 1]
F_minus: [2 2 1 4 4 1 4]
2 elements satisfy


array([3, 5])

In [4]:

def iterative_improvement(D, alphas, method):
    buffer = {"delta": [], "sigma": [], "loss": []}
    n = D.shape[0]
    
    def MRCI(n):
        table1 = [0, 0, 0.52, 0.89, 1.11, 1.25, 1.35, 1.40, 1.45, 1.49, 1.52, 1.54, 1.56, 1.58, 1.59]
        if n >= len(table1):
            raise ValueError("n too chonky")
        return table1[n-1]
    
    def CR(D):
        return ((np.linalg.eigvals(D)[0] - n) / (n - 1)) / MRCI(n)
    
    def get_eigvector(D):

        lambda_max = np.linalg.eigvals(D)[0]

        def objective(w):

            return np.linalg.norm(D @ w - lambda_max * w)

        bnds = tuple((0, None) for i in range(n))
        w0 = np.ones(n)
        result = minimize(objective, w0, bounds=bnds)
        return result["x"]

    def criterion_inf(D, D_updated):
        return np.linalg.norm(D - D_updated, ord=np.inf)

    def criterion_l2(D, D_updated):
        return np.linalg.norm(D - D_updated) / n

    for alpha in alphas:
        D_updated = D.copy()
        print(f"alpha={alpha}")

        buffer["delta"].append([])
        buffer["sigma"].append([])
        buffer["loss"].append([])
        counter = 0
        
        while CR(D_updated) > 0.1:
            counter += 1
            w = get_eigvector(D_updated)

            if method == "WGMM":
                D_updated = np.array(
                    [
                        [
                            ((D_updated[i, j]) ** alpha)
                            * ((w[i] / w[j])) ** (1 - alpha)
                            for j in range(n)
                        ]
                        for i in range(n)
                    ]
                )
                
            if method == "WAMM":
                D_updated = np.array(
                    [
                        [
                            ((D_updated[i, j]) * alpha) + ((w[i] / w[j])) * (1 - alpha)
                            if j > i
                            else 1.0
                            / (
                                ((D_updated[j, i]) * alpha)
                                + ((w[j] / w[i])) * (1 - alpha)
                            )
                            for j in range(n)
                        ]
                        for i in range(n)
                    ]
                )
                
            sigma = criterion_l2(D, D_updated)
            delta = criterion_inf(D, D_updated)
            loss = CR(D_updated)
            print(f"iteration {counter}")
            print(
                f"delta: {criterion_inf(D, D_updated)}, sigma: {criterion_l2(D, D_updated)}, loss: {loss}"
            )
            print(f"w:{np.round(w, 5)}")
            print(f"D: {np.round(D_updated, 3)}")

            buffer["delta"][-1].append(delta)
            buffer["sigma"][-1].append(sigma)
            buffer["loss"][-1].append(loss)

    print(len(buffer["delta"]))
    fig, axes = plt.subplots(3, 1, figsize=(12, 12))

    for deltas in buffer["delta"]:
        axes[0].plot(deltas)
        axes[0].set_title("delta")
        axes[0].legend(alphas)

    for sigmas in buffer["sigma"]:
        axes[1].plot(sigmas)
        axes[1].set_title("sigma")
        axes[1].legend(alphas)

    for losses in buffer["loss"]:
        axes[2].plot(losses)
        axes[2].set_title("loss")
        axes[2].legend(alphas)
        
    return D_updated

In [5]:
D_u = iterative_improvement(matrix, [0.999], "WAMM")

alpha=0.999
iteration 1
delta: 5.009999582470986, sigma: 0.7142389604149242, loss: (0.4537648695116715+0j)
w:[0.90443 0.90443 2.19867 1.19344 0.27852 1.52138 0.43949]
D: [[1.    1.    0.5   3.997 3.    0.334 2.   ]
 [1.    1.    0.5   3.997 3.    0.334 2.   ]
 [2.    2.    1.    7.994 6.002 0.501 4.001]
 [0.25  0.25  0.125 1.    1.003 6.994 0.502]
 [0.333 0.333 0.167 0.997 1.    0.111 1.   ]
 [2.998 2.998 1.996 0.143 8.994 1.    5.997]
 [0.5   0.5   0.25  1.991 1.    0.167 1.   ]]
iteration 2
delta: 5.017700646995159, sigma: 0.714254250415464, loss: (0.45275201874614596+0j)
w:[0.90862 0.90862 1.79435 1.19416 0.27907 1.48329 0.44043]
D: [[1.    1.    0.5   3.994 3.001 0.334 2.   ]
 [1.    1.    0.5   3.994 3.001 0.334 2.   ]
 [2.    2.    1.    7.987 6.002 0.502 4.001]
 [0.25  0.25  0.125 1.    1.007 6.988 0.504]
 [0.333 0.333 0.167 0.993 1.    0.111 0.999]
 [2.995 2.995 1.993 0.143 8.988 1.    5.995]
 [0.5   0.5   0.25  1.982 1.001 0.167 1.   ]]
iteration 3
delta: 5.025408644122842, si

iteration 20
delta: 5.157700569950443, sigma: 0.7157636769473161, loss: (0.4351647723550028+0j)
w:[0.91196 0.91196 1.80097 1.19251 0.27492 1.47479 0.42851]
D: [[1.    1.    0.5   3.936 3.006 0.339 2.002]
 [1.    1.    0.5   3.936 3.006 0.339 2.002]
 [2.    2.    1.    7.872 6.011 0.514 4.004]
 [0.254 0.254 0.127 1.    1.066 6.877 0.545]
 [0.333 0.333 0.166 0.938 1.    0.113 0.993]
 [2.951 2.951 1.944 0.145 8.88  1.    5.949]
 [0.5   0.5   0.25  1.837 1.007 0.168 1.   ]]
iteration 21
delta: 5.165571908993943, sigma: 0.7159114724689816, loss: (0.43422190386836934+0j)
w:[0.91214 0.91214 1.80133 1.19241 0.27471 1.47434 0.4279 ]
D: [[1.    1.    0.5   3.933 3.006 0.339 2.002]
 [1.    1.    0.5   3.933 3.006 0.339 2.002]
 [2.    2.    1.    7.865 6.012 0.515 4.004]
 [0.254 0.254 0.127 1.    1.069 6.871 0.547]
 [0.333 0.333 0.166 0.936 1.    0.113 0.992]
 [2.948 2.948 1.941 0.146 8.874 1.    5.946]
 [0.5   0.5   0.25  1.829 1.008 0.168 1.   ]]
iteration 22
delta: 5.173448795493457, sigma: 0.7

iteration 39
delta: 5.308140913213733, sigma: 0.7195867596756348, loss: (0.4178177087530067+0j)
w:[0.91529 0.91529 1.80761 1.19055 0.27109 1.46654 0.41763]
D: [[1.    1.    0.5   3.876 3.012 0.344 2.005]
 [1.    1.    0.5   3.876 3.012 0.344 2.005]
 [1.999 1.999 1.    7.752 6.022 0.528 4.009]
 [0.258 0.258 0.129 1.    1.128 6.763 0.587]
 [0.332 0.332 0.166 0.887 1.    0.114 0.986]
 [2.905 2.905 1.895 0.148 8.773 1.    5.902]
 [0.499 0.499 0.249 1.703 1.014 0.169 1.   ]]
iteration 40
delta: 5.316106493246752, sigma: 0.7198437268126617, loss: (0.4169365651444218+0j)
w:[0.91546 0.91546 1.80794 1.19044 0.2709  1.46613 0.4171 ]
D: [[1.    1.    0.5   3.873 3.012 0.345 2.005]
 [1.    1.    0.5   3.873 3.012 0.345 2.005]
 [1.999 1.999 1.    7.746 6.023 0.529 4.009]
 [0.258 0.258 0.129 1.    1.131 6.757 0.59 ]
 [0.332 0.332 0.166 0.884 1.    0.114 0.986]
 [2.903 2.903 1.892 0.148 8.767 1.    5.9  ]
 [0.499 0.499 0.249 1.696 1.014 0.169 1.   ]]
iteration 41
delta: 5.324076399167174, sigma: 0.72

 [0.498 0.498 0.249 1.58  1.021 0.171 1.   ]]
iteration 60
delta: 5.476246615156693, sigma: 0.7260125344493812, loss: (0.3999291403058782+0j)
w:[0.91877 0.91877 1.81452 1.18818 0.2674  1.45817 0.40729]
D: [[1.    1.    0.5   3.812 3.02  0.35  2.009]
 [1.    1.    0.5   3.812 3.02  0.35  2.009]
 [1.999 1.999 1.    7.622 6.037 0.543 4.017]
 [0.262 0.262 0.131 1.    1.196 6.639 0.635]
 [0.331 0.331 0.166 0.836 1.    0.115 0.979]
 [2.856 2.856 1.843 0.151 8.662 1.    5.853]
 [0.498 0.498 0.249 1.574 1.021 0.171 1.   ]]
iteration 61
delta: 5.484290531657452, sigma: 0.726369016883077, loss: (0.3991081976425653+0j)
w:[0.91893 0.91893 1.81484 1.18806 0.26723 1.45778 0.40683]
D: [[1.    1.    0.5   3.809 3.021 0.35  2.01 ]
 [1.    1.    0.5   3.809 3.021 0.35  2.01 ]
 [1.999 1.999 1.    7.616 6.038 0.543 4.017]
 [0.263 0.263 0.131 1.    1.199 6.634 0.637]
 [0.331 0.331 0.166 0.834 1.    0.116 0.979]
 [2.854 2.854 1.841 0.151 8.657 1.    5.851]
 [0.498 0.498 0.249 1.569 1.021 0.171 1.   ]]
itera

iteration 80
delta: 5.637651663379265, sigma: 0.7339110039989748, loss: (0.3840023660363477+0j)
w:[0.92188 0.92188 1.82072 1.18574 0.26434 1.4508  0.39882]
D: [[1.    1.    0.5   3.751 3.029 0.356 2.015]
 [1.    1.    0.5   3.751 3.029 0.356 2.015]
 [1.998 1.998 1.    7.502 6.053 0.557 4.027]
 [0.267 0.267 0.133 1.    1.261 6.524 0.681]
 [0.33  0.33  0.165 0.793 1.    0.117 0.973]
 [2.811 2.811 1.797 0.153 8.563 1.    5.809]
 [0.496 0.496 0.248 1.469 1.028 0.172 1.   ]]
iteration 81
delta: 5.645747322945705, sigma: 0.7343459378251068, loss: (0.3832321992179487+0j)
w:[0.92203 0.92203 1.82102 1.18561 0.2642  1.45045 0.39842]
D: [[1.    1.    0.5   3.749 3.03  0.356 2.015]
 [1.    1.    0.5   3.749 3.03  0.356 2.015]
 [1.998 1.998 1.    7.496 6.054 0.557 4.027]
 [0.267 0.267 0.133 1.    1.264 6.518 0.683]
 [0.33  0.33  0.165 0.791 1.    0.117 0.973]
 [2.809 2.809 1.794 0.153 8.558 1.    5.807]
 [0.496 0.496 0.248 1.464 1.028 0.172 1.   ]]
iteration 82
delta: 5.653845025180997, sigma: 0.73

...

iteration 698
delta: 10.088373354095957, sigma: 1.1810199246762725, loss: (0.13309375168725418+0j)
w:[1.02099 1.02099 2.01707 1.14829 0.25602 1.4083  0.35387]
D: [[1.    1.    0.503 2.41  3.414 0.511 2.347]
 [1.    1.    0.503 2.41  3.414 0.511 2.347]
 [1.988 1.988 1.    4.81  6.782 0.931 4.661]
 [0.415 0.415 0.208 1.    2.796 3.895 1.864]
 [0.293 0.293 0.147 0.358 1.    0.146 0.85 ]
 [1.957 1.957 1.074 0.257 6.869 1.    4.949]
 [0.426 0.426 0.215 0.537 1.176 0.202 1.   ]]
iteration 699
delta: 10.094088671345677, sigma: 1.1817046781158747, loss: (0.13288133507700434+0j)
w:[1.02109 1.02109 2.01728 1.14822 0.25604 1.40827 0.35388]
D: [[1.    1.    0.503 2.408 3.415 0.511 2.347]
 [1.    1.    0.503 2.408 3.415 0.511 2.347]
 [1.988 1.988 1.    4.806 6.783 0.931 4.662]
 [0.415 0.415 0.208 1.    2.798 3.892 1.865]
 [0.293 0.293 0.147 0.357 1.    0.146 0.85 ]
 [1.956 1.956 1.074 0.257 6.867 1.    4.949]
 [0.426 0.426 0.215 0.536 1.176 0.202 1.   ]]
iteration 700
delta: 10.099798887161034, sig

iteration 717
delta: 10.196094278236089, sigma: 1.193945111129548, loss: (0.1291178477878303+0j)
w:[1.02299 1.02299 2.02102 1.14684 0.25635 1.40782 0.35406]
D: [[1.    1.    0.503 2.381 3.425 0.515 2.357]
 [1.    1.    0.503 2.381 3.425 0.515 2.357]
 [1.988 1.988 1.    4.752 6.802 0.94  4.68 ]
 [0.42  0.42  0.21  1.    2.828 3.837 1.889]
 [0.292 0.292 0.147 0.354 1.    0.146 0.848]
 [1.941 1.941 1.063 0.261 6.837 1.    4.931]
 [0.424 0.424 0.214 0.529 1.179 0.203 1.   ]]
iteration 718
delta: 10.201713066884771, sigma: 1.1946203882543416, loss: (0.12891205611953388+0j)
w:[1.02309 1.02309 2.02123 1.14677 0.25637 1.40779 0.35407]
D: [[1.    1.    0.503 2.38  3.426 0.515 2.357]
 [1.    1.    0.503 2.38  3.426 0.515 2.357]
 [1.987 1.987 1.    4.749 6.804 0.941 4.681]
 [0.42  0.42  0.211 1.    2.829 3.834 1.891]
 [0.292 0.292 0.147 0.353 1.    0.146 0.848]
 [1.94  1.94  1.063 0.261 6.835 1.    4.93 ]
 [0.424 0.424 0.214 0.529 1.179 0.203 1.   ]]
iteration 719
delta: 10.207326799038512, sigma

w:[1.02493 1.02493 2.02488 1.14539 0.25667 1.40733 0.35426]
D: [[1.    1.    0.503 2.353 3.436 0.519 2.367]
 [1.    1.    0.503 2.353 3.436 0.519 2.367]
 [1.987 1.987 1.    4.696 6.823 0.95  4.7  ]
 [0.425 0.425 0.213 1.    2.858 3.78  1.915]
 [0.291 0.291 0.147 0.35  1.    0.147 0.846]
 [1.926 1.926 1.053 0.265 6.805 1.    4.913]
 [0.422 0.422 0.213 0.522 1.182 0.204 1.   ]]
iteration 737
delta: 10.307512159351525, sigma: 1.2073551340786703, loss: (0.12506623996249716+0j)
w:[1.02503 1.02503 2.02508 1.14532 0.25669 1.4073  0.35427]
D: [[1.    1.    0.503 2.352 3.436 0.519 2.368]
 [1.    1.    0.503 2.352 3.436 0.519 2.368]
 [1.987 1.987 1.    4.693 6.824 0.95  4.701]
 [0.425 0.425 0.213 1.    2.86  3.777 1.916]
 [0.291 0.291 0.147 0.35  1.    0.147 0.846]
 [1.926 1.926 1.052 0.265 6.804 1.    4.912]
 [0.422 0.422 0.213 0.522 1.183 0.204 1.   ]]
iteration 738
delta: 10.313030294466056, sigma: 1.2080203420071192, loss: (0.12486716369042841+0j)
w:[1.02514 1.02514 2.02528 1.14524 0.25671 1

iteration 756
delta: 10.411503302846018, sigma: 1.219907489637231, loss: (0.12133945740754054+0j)
w:[1.02693 1.02693 2.02883 1.14386 0.25702 1.4068  0.35448]
D: [[1.    1.    0.503 2.324 3.447 0.523 2.377]
 [1.    1.    0.503 2.324 3.447 0.523 2.377]
 [1.987 1.987 1.    4.638 6.844 0.959 4.72 ]
 [0.43  0.43  0.216 1.    2.89  3.721 1.941]
 [0.29  0.29  0.146 0.346 1.    0.148 0.843]
 [1.911 1.911 1.042 0.269 6.773 1.    4.895]
 [0.421 0.421 0.212 0.515 1.186 0.204 1.   ]]
iteration 757
delta: 10.416926782895143, sigma: 1.220563066743561, loss: (0.12114652742160499+0j)
w:[1.02703 1.02703 2.02902 1.14378 0.25703 1.40677 0.35449]
D: [[1.    1.    0.503 2.323 3.447 0.523 2.378]
 [1.    1.    0.503 2.323 3.447 0.523 2.378]
 [1.987 1.987 1.    4.635 6.845 0.96  4.721]
 [0.43  0.43  0.216 1.    2.892 3.718 1.942]
 [0.29  0.29  0.146 0.346 1.    0.148 0.843]
 [1.91  1.91  1.042 0.269 6.771 1.    4.894]
 [0.421 0.421 0.212 0.515 1.186 0.204 1.   ]]
iteration 758
delta: 10.42234530784376, sigma:

KeyboardInterrupt: 